# Benchmark the reliability solvers on the problems

In this example, we show how to run all the methods on all the problems and get the computed probability.

In [1]:
import openturns as ot
import numpy as np
import otbenchmark as otb
import pandas as pd
import csv

We import the list of reliability problems.

In [2]:
benchmarkProblemList = otb.ReliabilityBenchmarkProblemList()
numberOfProblems = len(benchmarkProblemList)
numberOfProblems

26

In [3]:
for i in range(numberOfProblems):
    problem = benchmarkProblemList[i]
    name = problem.getName()
    pf = problem.getProbability()
    print("#", i, " : ", name, ", exact PF : ", pf)

# 0  :  RP8 , exact PF :  0.000784
# 1  :  RP14 , exact PF :  0.00752
# 2  :  RP22 , exact PF :  0.00416
# 3  :  RP24 , exact PF :  0.00286
# 4  :  RP25 , exact PF :  6.14e-06
# 5  :  RP28 , exact PF :  1.46e-07
# 6  :  RP31 , exact PF :  0.00018
# 7  :  RP33 , exact PF :  0.00257
# 8  :  RP35 , exact PF :  0.00354
# 9  :  RP38 , exact PF :  0.0081
# 10  :  RP53 , exact PF :  0.0313
# 11  :  RP55 , exact PF :  0.36
# 12  :  RP54 , exact PF :  0.000998
# 13  :  RP57 , exact PF :  0.0284
# 14  :  RP75 , exact PF :  0.0107
# 15  :  RP89 , exact PF :  0.00543
# 16  :  RP107 , exact PF :  2.92e-07
# 17  :  RP110 , exact PF :  3.19e-05
# 18  :  RP111 , exact PF :  7.65e-07
# 19  :  RP63 , exact PF :  0.000379
# 20  :  RP91 , exact PF :  0.000697
# 21  :  RP60 , exact PF :  0.0456
# 22  :  RP77 , exact PF :  2.87e-07
# 23  :  Four-branch serial system (Waarts, 2000) , exact PF :  0.0021859614549132322
# 24  :  R-S , exact PF :  0.07864960352514257
# 25  :  Axial stressed beam , exact PF :  0.

In [4]:
maximumEvaluationNumber=1000
maximumAbsoluteError=1.0e-3
maximumRelativeError=1.0e-3
maximumResidualError=1.0e-3
maximumConstraintError=1.0e-3
nearestPointAlgorithm = ot.AbdoRackwitz()
nearestPointAlgorithm.setMaximumEvaluationNumber(maximumEvaluationNumber)
nearestPointAlgorithm.setMaximumAbsoluteError(maximumAbsoluteError)
nearestPointAlgorithm.setMaximumRelativeError(maximumRelativeError)
nearestPointAlgorithm.setMaximumResidualError(maximumResidualError)
nearestPointAlgorithm.setMaximumConstraintError(maximumConstraintError)

In [5]:
i = 3
problem = benchmarkProblemList[i]
metaAlgorithm = otb.ReliabilityBenchmarkMetaAlgorithm(problem)

In [6]:
benchmarkFORM = metaAlgorithm.runFORM(nearestPointAlgorithm)
s1 = benchmarkFORM.summary()
print(s1)

computedProbability = 0.006209245091320783  exactProbability = 0.00286  absoluteError = 0.003349245091320783 numberOfCorrectDigits = -0.06858089640947408 numberOfFunctionEvaluations = 5


In [7]:
benchmarkSORM = metaAlgorithm.runSORM(nearestPointAlgorithm)
s2 = benchmarkSORM.summary()
print(s2)

computedProbability = 0.006209245091320783  exactProbability = 0.00286  absoluteError = 0.003349245091320783 numberOfCorrectDigits = -0.06858089640947408 numberOfFunctionEvaluations = 5


In [8]:
benchmarkMC = metaAlgorithm.runMonteCarlo(
    maximumOuterSampling=1000000, coefficientOfVariation=0.1, blockSize=1,
)
s3 = benchmarkMC.summary()
print(s3)

computedProbability = 0.002858694719990857  exactProbability = 0.00286  absoluteError = 1.3052800091430984e-06 numberOfCorrectDigits = 3.340662346449938 numberOfFunctionEvaluations = 34981


In [9]:
benchmarkFORMIS = metaAlgorithm.runFORMImportanceSampling(
    nearestPointAlgorithm, 
    maximumOuterSampling=5000,
    coefficientOfVariation=0.1,
)
s4 = benchmarkFORMIS.summary()
print(s4)

computedProbability = 0.0028912458245778103  exactProbability = 0.00286  absoluteError = 3.124582457781017e-05 numberOfCorrectDigits = 1.9615740429360635 numberOfFunctionEvaluations = 601


In [10]:
benchmarkSS = metaAlgorithm.runSubsetSampling(
    maximumOuterSampling=5000, coefficientOfVariation=0.1, blockSize=1,
)
s5 = benchmarkSS.summary()
print(s5)

computedProbability = 0.002817999999999997  exactProbability = 0.00286  absoluteError = 4.200000000000297e-05 numberOfCorrectDigits = 1.8331167427311117 numberOfFunctionEvaluations = 15000


In [11]:
problem_names = []
for i in range(numberOfProblems):
    problem = benchmarkProblemList[i]
    name = problem.getName()
    problem_names.append(name)

In [12]:
metrics = [
    "Exact",
    "FORM",
    "SORM",
    "Monte Carlo",
    "FORM-IS",
    "Subset",
]
results = np.zeros((numberOfProblems, len(metrics)))

for i in range(numberOfProblems):
    problem = benchmarkProblemList[i]
    results[i][0] = problem.getProbability()
    metaAlgorithm = otb.ReliabilityBenchmarkMetaAlgorithm(problem)
    benchmarkResult = metaAlgorithm.runFORM(nearestPointAlgorithm)
    results[i][1] = benchmarkResult.computedProbability
    benchmarkResult = metaAlgorithm.runSORM(nearestPointAlgorithm)
    results[i][2] = benchmarkResult.computedProbability
    benchmarkResult = metaAlgorithm.runMonteCarlo(
        maximumOuterSampling=10000, coefficientOfVariation=0.1, blockSize=1,
    )
    results[i][3] = benchmarkResult.computedProbability
    benchmarkResult = metaAlgorithm.runFORMImportanceSampling(
        nearestPointAlgorithm
    )
    results[i][4] = benchmarkResult.computedProbability
    benchmarkResult = metaAlgorithm.runSubsetSampling(
        maximumOuterSampling=5000, coefficientOfVariation=0.1, blockSize=1,
    )
    results[i][5] = benchmarkResult.computedProbability

df = pd.DataFrame(results, index=problem_names, columns=metrics)
df

,Exact,FORM,SORM,Monte Carlo,FORM-IS,Subset
RP8,7.840000e-04,6.598878e-04,7.838036e-04,0.000700,7.099150e-04,9.014000e-04
RP14,7.520000e-03,7.003011e-04,6.995436e-04,0.000700,6.418774e-04,6.956060e-04
RP22,4.160000e-03,6.209672e-03,4.390902e-03,0.005200,4.329853e-03,4.188000e-03
RP24,2.860000e-03,6.209245e-03,6.209245e-03,0.003100,2.690704e-03,2.528000e-03
RP25,6.140000e-06,6.349316e-03,6.349316e-03,0.000000,3.782391e-05,3.536000e-05
RP28,1.460000e-07,2.850470e-08,0.000000e+00,0.000000,1.271001e-07,1.215764e-07
RP31,1.800000e-04,2.275013e-02,2.275013e-02,0.002800,3.614201e-03,3.660000e-03
RP33,2.570000e-03,1.349898e-03,1.349898e-03,0.002200,1.856178e-03,2.642704e-03
RP35,3.540000e-03,1.349898e-03,2.134376e-03,0.003400,2.428236e-03,3.558000e-03
RP38,8.100000e-03,7.902212e-03,8.029356e-03,0.007900,6.695971e-03,7.834000e-03


In [14]:
df.to_csv("reliability_benchmark_table-output.csv")